In [8]:
import json
import os
import re
from datetime import datetime
import unicodedata
from typing import List, Dict, Any

In [9]:
# ===============================
# 1️⃣ Làm sạch văn bản
# ===============================
def clean_formatting(text: str) -> str:
    if not text:
        return ""
    text = text.replace('\r', '\n')
    text = re.sub(r"(?m)^[\*\=\-–_\.]{3,}\s*$", "", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)
    text = re.sub(r"([^\n])\n([^\n])", r"\1 \2", text)
    text = re.sub(r"KHÔNGSỐ", "KHÔNG SỐ", text, flags=re.IGNORECASE)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# ===============================
# 2️⃣ Suy luận lĩnh vực (dự phòng)
# ===============================
def infer_field_category(title: str | None, agency: str | None, content: str | None) -> str | None:
    combined = " ".join(filter(None, [title, agency, content])).lower()
    domain_keywords = {
        "giáo dục": ["giáo dục", "trường học", "học sinh", "đại học", "bộ giáo dục"],
        "y tế": ["y tế", "bệnh viện", "sức khỏe", "bộ y tế"],
        "tài chính": ["tài chính", "ngân sách", "thuế", "kho bạc", "bộ tài chính"],
        "đất đai": ["đất đai", "bất động sản", "xây dựng", "bộ xây dựng"],
        "lao động": ["lao động", "việc làm", "tiền lương", "bộ lao động"],
        "nông nghiệp": ["nông nghiệp", "thủy sản", "chăn nuôi", "bộ nông nghiệp"],
        "giao thông": ["giao thông", "vận tải", "đường bộ", "bộ giao thông"],
        "công thương": ["công thương", "thương mại", "xuất nhập khẩu", "bộ công thương"],
        "quốc phòng": ["quốc phòng", "quân đội", "bộ quốc phòng"],
        "công an": ["công an", "an ninh", "bộ công an"],
        "văn hóa": ["văn hóa", "thể thao", "du lịch", "bộ văn hóa"],
        "môi trường": ["môi trường", "tài nguyên", "bộ tài nguyên"],
    }
    for field, keywords in domain_keywords.items():
        if any(kw in combined for kw in keywords):
            return field.capitalize()
    return "Khác"

In [10]:
# ===============================
# 4️⃣ Chia văn bản thành các chunk
# ===============================
def chunk_document(content: str, max_chunk_size: int = 1000, overlap_words: int = 50) -> List[str]:
    """Chia văn bản thành các đoạn nhỏ dựa trên câu, điều luật, chương,..."""
    content = clean_formatting(content)

    # --- Tách theo các chỉ mục pháp lý (Điều, Khoản, Chương, Mục)
    parts = re.split(r'(?=(Điều\s+thứ|KHOẢN\s+THỨ|CHƯƠNG\s+|MỤC\s+))', content, flags=re.IGNORECASE)
    if len(parts) == 1:
        # Nếu không có 'Điều...', chia theo câu
        sentences = re.split(r'(?<=[.!?])\s+', content)
        parts = []
        current = ""
        for s in sentences:
            if len(current) + len(s) < max_chunk_size:
                current += " " + s
            else:
                parts.append(current.strip())
                current = s
        if current:
            parts.append(current.strip())
    else:
        merged = []
        for i in range(0, len(parts), 2):
            section = "".join(parts[i:i+2]).strip()
            if section:
                merged.append(section)
        parts = merged

    # --- Gộp nhỏ nếu đoạn quá ngắn
    chunks = []
    buffer = ""
    for p in parts:
        if len(buffer) + len(p) < max_chunk_size:
            buffer += " " + p
        else:
            chunks.append(buffer.strip())
            buffer = p
    if buffer:
        chunks.append(buffer.strip())

    # --- Overlap giữa các chunk
    if overlap_words > 0 and len(chunks) > 1:
        result = [chunks[0]]
        for i in range(1, len(chunks)):
            prev = chunks[i-1].split()
            overlap = " ".join(prev[-overlap_words:]) if len(prev) > overlap_words else chunks[i-1]
            result.append((overlap + " " + chunks[i]).strip())
        chunks = result

    return chunks

# ===============================
# 5️⃣ Chuẩn hóa từng record
# ===============================
def normalize_record(doc: Dict[str, Any]) -> Dict[str, Any]:
    content = doc.get("mt6_text") or doc.get("content") or ""
    content = clean_formatting(content)
    law_id = str(doc.get("law_id") or doc.get("lawid") or "unknown")
    return {
        "law_id": law_id,
        "raw_title": doc.get("title"),
        "source_url": doc.get("href"),
        "category": doc.get("category"),
        "content": content,
        "provided_issue_date": doc.get("date")
    }

In [11]:
# ===============================
# 3️⃣ Trích xuất metadata
# ===============================
def extract_metadata(content: str, law_id: str, title: str | None = None) -> Dict[str, Any]:
    metadata = {
        "law_id": law_id,
        "document_type": None,
        "issuing_agency": None,
        "issue_date": None,
        "title": None,
    }

    DOC_TYPES = [
        "CÔNG ĐIỆN", "NGHỊ ĐỊNH", "THÔNG TƯ", "QUYẾT ĐỊNH", "LUẬT",
        "NGHỊ QUYẾT", "CHỈ THỊ", "THÔNG BÁO", "KẾ HOẠCH", "BÁO CÁO",
        "VĂN BẢN", "CÔNG VĂN", "SẮC LỆNH"
    ]
    AGENCIES = [
        "CHỦ TỊCH", "CHÍNH PHỦ", "BỘ", "QUỐC HỘI", "THỦ TƯỚNG",
        "TÒA ÁN", "VIỆN KIỂM SÁT", "KIỂM TOÁN", "UBND", "HĐND"
    ]

    def _norm(s: str) -> str:
        s = unicodedata.normalize("NFC", s or "").lower()
        return re.sub(r"\s+", " ", s).strip()

    # 1️⃣ Loại văn bản
    if title:
        tnorm = _norm(title)
        for dt in DOC_TYPES:
            if dt.lower() in tnorm:
                metadata["document_type"] = dt
                break
    if not metadata["document_type"]:
        head = "\n".join(content.splitlines()[:20]).upper()
        for dt in DOC_TYPES:
            if re.search(rf"\b{dt}\b", head, re.IGNORECASE):
                metadata["document_type"] = dt
                break

    # 2️⃣ Cơ quan ban hành
    if title:
        tnorm = _norm(title)
        if "chủ tịch" in tnorm:
            metadata["issuing_agency"] = "CHỦ TỊCH NƯỚC"
        elif "bộ" in tnorm:
            m = re.search(r"bộ\s+([^\d,.;]+)", tnorm)
            if m:
                metadata["issuing_agency"] = "BỘ " + m.group(1).upper()
        elif "chính phủ" in tnorm:
            metadata["issuing_agency"] = "CHÍNH PHỦ"
    if not metadata["issuing_agency"]:
        scan = content[:800].upper()
        for ag in AGENCIES:
            if ag in scan:
                metadata["issuing_agency"] = ag
                break

    # 3️⃣ Ngày ban hành
    date_patterns = [
        r'ngày\s+(\d{1,2})\s+tháng\s+(\d{1,2})\s+năm\s+(\d{4})',
        r'(\d{1,2})[./-](\d{1,2})[./-](\d{4})'
    ]
    for pat in date_patterns:
        m = re.search(pat, content, re.IGNORECASE)
        if m:
            try:
                d, mth, y = int(m.group(1)), int(m.group(2)), int(m.group(3))
                metadata["issue_date"] = datetime(y, mth, d).strftime("%Y-%m-%d")
                break
            except ValueError:
                continue

    # 4️⃣ Tiêu đề
    metadata["title"] = title.strip() if title else None
    return metadata


In [12]:
# ===============================
# 4️⃣ Chunking
# ===============================
def chunk_document(content: str, max_chunk_size: int = 1000, overlap_words: int = 50) -> List[str]:
    content = clean_formatting(content)
    parts = re.split(r'(?=(Điều\s+thứ|KHOẢN\s+THỨ|CHƯƠNG\s+|MỤC\s+))', content, flags=re.IGNORECASE)

    if len(parts) == 1:
        sentences = re.split(r'(?<=[.!?])\s+', content)
        parts = []
        current = ""
        for s in sentences:
            if len(current) + len(s) < max_chunk_size:
                current += " " + s
            else:
                parts.append(current.strip())
                current = s
        if current:
            parts.append(current.strip())
    else:
        merged = []
        for i in range(0, len(parts), 2):
            section = "".join(parts[i:i+2]).strip()
            if section:
                merged.append(section)
        parts = merged

    chunks = []
    buffer = ""
    for p in parts:
        if len(buffer) + len(p) < max_chunk_size:
            buffer += " " + p
        else:
            chunks.append(buffer.strip())
            buffer = p
    if buffer:
        chunks.append(buffer.strip())

    if overlap_words > 0 and len(chunks) > 1:
        result = [chunks[0]]
        for i in range(1, len(chunks)):
            prev = chunks[i-1].split()
            overlap = " ".join(prev[-overlap_words:]) if len(prev) > overlap_words else chunks[i-1]
            result.append((overlap + " " + chunks[i]).strip())
        chunks = result

    return chunks

# ===============================
# 5️⃣ Chuẩn hóa bản ghi
# ===============================
def normalize_record(doc: Dict[str, Any]) -> Dict[str, Any]:
    """Dữ liệu gốc: content = category, mt6_text = full text"""
    category = doc.get("content") or ""
    content = doc.get("mt6_text") or ""
    content = clean_formatting(content)
    law_id = str(doc.get("law_id") or doc.get("lawid") or "unknown")
    return {
        "law_id": law_id,
        "raw_title": doc.get("title"),
        "source_url": doc.get("href"),
        "category": category.strip(),
        "content": content,
        "provided_issue_date": doc.get("date")
    }



In [13]:
# ===============================
# 6️⃣ Pipeline chính
# ===============================
def process_law_data(input_path: str, max_chunk_size: int = 400, overlap_words: int = 30) -> str:
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Không tìm thấy file: {input_path}")

    base = os.path.splitext(os.path.basename(input_path))[0]
    output_dir = os.path.dirname(input_path) or "."
    processed_path = os.path.join(output_dir, f"{base}_processedv6.json")

    with open(input_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    if isinstance(raw_data, dict):
        raw_data = [raw_data]

    processed: List[Dict[str, Any]] = []

    for doc in raw_data:
        norm = normalize_record(doc)
        metadata = extract_metadata(norm["content"], norm["law_id"], norm.get("raw_title"))
        chunks = chunk_document(norm["content"], max_chunk_size=max_chunk_size, overlap_words=overlap_words)

        # ✅ Ưu tiên category từ dữ liệu gốc
        if norm["category"]:
            final_category = norm["category"]
        else:
            final_category = infer_field_category(norm["raw_title"], metadata["issuing_agency"], norm["content"])

        for chunk_num, chunk_text in enumerate(chunks, start=1):
            record = {
                "law_id": norm["law_id"],
                "chunk_num": chunk_num,
                "document_type": metadata["document_type"],
                "issuing_agency": metadata["issuing_agency"],
                "issue_date": metadata["issue_date"] or norm.get("provided_issue_date"),
                "title": metadata["title"],
                "source_url": norm["source_url"],
                "raw_title": norm["raw_title"],
                "category": final_category,
                "chunk": chunk_text
            }
            record = {k: v for k, v in record.items() if v not in [None, ""]}
            processed.append(record)


    with open(processed_path, "w", encoding="utf-8") as f:
        json.dump(processed, f, ensure_ascii=False, indent=2)

    return processed_path


In [14]:
# ===============================
# 7️⃣ Gọi thử pipeline
# ===============================
input_path = r"C:\Code\Ky5\SEG\Crawl\Dataprocessing\Process\tvpl_congvan3days.json"
output_path = process_law_data(input_path)

print("✅ Đã xử lý xong!")
print(f"📄 File đầu ra: {output_path}")

with open(output_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
print(f"📦 Tổng số record (chunk): {len(data)}")
print("\n--- Ví dụ record đầu tiên ---")
print(json.dumps(data[0], ensure_ascii=False, indent=2))

✅ Đã xử lý xong!
📄 File đầu ra: C:\Code\Ky5\SEG\Crawl\Dataprocessing\Process\tvpl_congvan3days_processedv6.json
📦 Tổng số record (chunk): 374251

--- Ví dụ record đầu tiên ---
{
  "law_id": "22714",
  "chunk_num": 1,
  "document_type": "NGHỊ ĐỊNH",
  "issuing_agency": "BỘ TRƯỞNG BỘ QUỐC GIA GIÁO DỤC BAN HÀNH",
  "issue_date": "1945-10-15",
  "title": "Nghị định năm 1945 về Hội đồng cố vấn học chính do Bộ trưởng Bộ Quốc Gia Giáo Dục ban hành.",
  "source_url": "https://thuvienphapluat.vn/van-ban/Giao-duc/Nghi-dinh-Hoi-dong-co-van-hoc-chinh-22714.aspx",
  "raw_title": "Nghị định năm 1945 về Hội đồng cố vấn học chính do Bộ trưởng Bộ Quốc Gia Giáo Dục ban hành.",
  "category": "Giáo dục",
  "chunk": "BỘ QUỐC GIA GIÁO DỤC VIỆT NAM DÂN CHỦ CỘNG HÒA Độc lập - Tự do - Hạnh phúc Số: KHÔNG SỐ1 Hà Nội, ngày 15 tháng 10 năm 1945 BỘ TRƯỞNG BỘ QUỐC GIA GIÁO DỤC Chiếu chỉ Sắc lệnh số 44 ngày 10 tháng 10 năm 1945 thiết lập Hội đồng cố vấn Học chính. NGHỊ ĐỊNH: Điều thứ"
}
